# Impact VM

This file simulates the Impact Virtual Machine, which executes the (public) transcripts emitted by the compactc compiler on the Midnight ledger. 

The intention of this file is to simulate the on-chain runtime as defined here: https://github.com/input-output-hk/midnight-architecture/tree/main/apis-and-common-types/onchain-runtime

In [1]:
# Bring in ledger ADTs
%run ./LedgerADTs.ipynb

## Design

Impact is a stack-based virtual Machine (NOTE: Thomas mentioned that some aspects, such as it's internal state, of the VM will be redesigned in the near-ish future). 

We emulate the VM through a (shallow) embedding of transformations of its internal state, i.e., the stack. An execution on the VM is given by a Python object of type `Callable[Stack,Stack]` that describes the execution's effect on the stack. 

For example: 

```python
def f(s): 
  s.PUSH(1)
   .PUSH(1)
   .ADD()

# gives `[2]` 
f(Stack()) 

# gives `3`
f(Stack()).cost() 
```



In [24]:
from typing import Any

class Stack(): 
    _stack : List[Any]
    # Think of this as the amount of gas spent to arrive at the current stack
    _cost  : int 
    
    def __init__(self) -> None:
        self._stack = list()
        self._cost = 0 

    def cost(self) -> int: 
        return self._cost

    def NOOP(self, n : int): 
        self._cost += n 
        return self
    
    def EQ(self): 
        b = self._stack.pop()
        a = self._stack.pop()
        self._stack.append(a == b)
        self._cost += 1
        return self

    def LT(self): 
        b = self._stack.pop()
        a = self._stack.pop() 
        self._stack.append(a < b) 
        self._cost += 1
        return self

    def TYPE(self): 
        # Replaces the value at the top of the stack with its type
        self._cost += 1
        return self

    def SIZE(self):
        # Replaces the value at the top of the stack with its size
        self._cost += 1
        return self

    def NEW(self):
        # Not entirely sure what this does? 
        self._cost += 1
        return self

    def AND(self): 
        b = self._stack.pop()
        a = self._stack.pop()
        self._stack.append(a and b)
        self._cost += 1
        return self

    def OR(self): 
        a = self._stack.pop()
        b = self._stack.pop()
        self._stack.append(a or b)
        self._cost += 1
        return self

    def NEG(self): 
        a = self._stack.pop() 
        self._stack.append(not a)
        self._cost += 1
        return self

    def LOG(self): 
        print(self._stack.pop())
        return self

    def ROOT(self): 
        a = self._stack.pop() 
        self._stack.append(a.root())
        return self

    def POP(self): 
        self._stack.pop() 
        return self
    
    def PUSH(self, a): 
        self._stack.append(a)
        return self

    def ADD(self): 
        b = self._stack.pop()
        a = self._stack.pop()
        self._stack.append(a + b)
        self._cost += 1
        return self

## Example

In [23]:
def prog(s : Stack) -> Stack: 
    return ( s
      .PUSH(1)
      .PUSH(1)
      .ADD()
      )

x = Stack()
print(prog(x)._stack)


[2]
